In [ ]:
from dolfin import *
import numpy as np
from numpy.random import random



# Sub domain for Periodic boundary condition
class PeriodicBoundary(SubDomain):

    # Left boundary is "target domain" G
    def inside(self, x, on_boundary):
        # return True if on left or bottom boundary AND NOT on one of the two corners (0, 1) and (1, 0)
        return bool((near(x[0], 0) or near(x[1], 0)) and \
                (not ((near(x[0], 0) and near(x[1], 1)) or \
                        (near(x[0], 1) and near(x[1], 0)))) and on_boundary)





    # Map right boundary (H) to left boundary (G)
    def map(self, x, y):
        if near(x[0], 1) and near(x[1], 1):
            y[0] = x[0] - 1.
            y[1] = x[1] - 1.
        elif near(x[0], 1):
            y[0] = x[0] - 1.
            y[1] = x[1]
        else:   # near(x[1], 1)
            y[0] = x[0]
            y[1] = x[1] - 1.

#Initial condtions (Random)
class IC(UserExpression):
    def eval(self,values,x):
        values[0] = 0.1*random() -0.1*random()
        values[1] = 0.1*random() -0.1*random()
    def value_shape(self):
        return(2,)
    



#Parameters
G = Constant(0.9)
A = Constant(6.0)
B = Constant(4.0)
d = Constant(20.0)
dt = 1.0e-06

# Form compiler options
parameters["form_compiler"]["optimize"]     = True
parameters["form_compiler"]["cpp_optimize"] = True
#parameters["form_compiler"]["representation"] = "quadrature"

# Create mesh and finite element
mesh = UnitSquareMesh(30, 30)
P1 = FiniteElement("P", triangle, 1)
element = MixedElement([P1, P1])
V = FunctionSpace(mesh, element)
plot(mesh)

# Condensed definition and encapsulation of functions
du       =TrialFunction(V)
v_1, v_2 = TestFunctions(V)
u        =Function(V) #current time-step
u_n      = Function(V) #previous time step

# Split system functions to access components
u_1,u_2=split(u)
u_n1, u_n2 = split(u_n)

#Initializzatioin
u1init = IC(element = u_n.ufl_element())
u.interpolate(u1init)
u_n.interpolate(u1init)

def ff(u_n1,u_n2):
    return u_n1-A*u_n2+G*u_n1*u_n2-u_n1**3

def gg(u_n1,u_n2):
    return u_n1-B*u_n2



# Forcing terms (x[0], x[1] are symbolic spatial variables, not (u,v) field variables)

"""ff = Expression('x[0] - A*x[1] + G*x[0]*x[1] - pow(x[0],3)', A=Constant(6.0), G=Constant(0.9),degree=1)
gg = Expression('x[0] - B*x[1]', B=Constant(4.0),degree=1)"""



# Define variational problem
F =  (u_1*v_1/dt)*dx \
    + (u_2*v_2/dt)*dx \
    + inner(grad(u_1), grad(v_1))*dx \
    + d*inner(grad(u_2), grad(v_2))*dx \
    -(dot(u_n1,v_1)/dt + dot(u_n2,v_2)/dt)*dx \
    -(dot(ff(u_n1,u_n2),v_1) + dot(gg(u_n1,u_n2),v_2))*dx 




#Time parameters:
t = 0.0
T=600*dt
i=0;
#vtk file
vtkfile_u_1 = File("reaction_system/u_1.pvd","compressed")
vtkfile_u_2 = File("reaction_system/u_2.pvd","compressed")

#Jacobian, gateau derivative direction (du)

J=derivative(F,u,du)




problem = NonlinearVariationalProblem(F, u, J=J)

"""M = inner(u_1, u_1)*dx
epsilon_M = 1e-7
solver = AdaptiveNonlinearVariationalSolver(problem, M)"""

#For solving the NonLinear Problem using "snes+gmres+hypre_amg"
solver = NonlinearVariationalSolver(problem)

prm = solver.parameters
prm['nonlinear_solver'] = 'snes'
prm['snes_solver']['line_search'] = 'bt'
#prm['snes_solver']['linear_solver'] = 'mumps'
prm['snes_solver']['linear_solver'] = 'gmres'
prm['snes_solver']['preconditioner'] = 'hypre_amg'
#prm['snes_solver']['krylov_solver']['nonzero_initial_guess'] = False
prm['snes_solver']['report'] = True

while (t<T):
# Update current time
    i +=1
    t += dt
    u_n.assign(u)
    solver.solve()
    #solver.solve(epsilon_M)
    #solve(F==0, u,J=J) 
    diff = u.vector() - u_n.vector() 
    eps = np.linalg.norm(diff, ord=np.Inf) #infinity norm
    print ("iter=%d: norm=%g" % (i, eps))
# Save solution to file (VTK)
    _u_1, _u_2 = u.split()
    vtkfile_u_1 << (_u_1, t)
    vtkfile_u_2 << (_u_2, t)
    
plot(u_1)


